In [ ]:
import graphAlgorithms as ga

# Load & Pre-Process Networks

load data set 

you can store your networks in any common format 

the graphAlgorithms package requires that the networks are provided as NetworkX Graph objects - refer to its documentation for detailed instructions

the networks should be weighted, if you have an unweighted network then assign all edges the same edge weight

the package assumes "weight" to be the default edge weight label, but this can be set when needed

In [ ]:
#location where the raw data files are stored, it is set to run from the installation folder
#- if applicable please change or CHANGE to the location of your networks

graph_location = "../networks/edgelists/"

In [ ]:
#location where output should be saved
#Please set location
location = ""

below an example on how to load an Edgelist with column headings into a NetworkX Graph object

There are multiple examples on how to load different formats in the import and export networks notebook

In [ ]:
import glob
import pandas as pd
import networkx as nx
import numpy as np

In [ ]:

labels = []
networks_graphs = []

print("load networks")
#gets all files located in the specified folder that end on .edgelist
#CHANGE the ending if your files end differently
for path in glob.glob(graph_location +"*.edgelist"):
    
    #you can specify that only part of the file name should be used as network name for later identification
    name =  path.split("/")[-1].replace(".rds.edgelist", "")
    
    
    #read the edgelist file as a dataframe
    fh = pd.read_csv(path, sep="\t")
    #convert it into a NetworkX graph G and specify the column names of the node pairs
    G=nx.from_pandas_edgelist(fh, "V1", "V2")
    
    #if you have an unweighted network assign all edges the same edge weight - here a value of 1 is assigned
    for u, v, d in G.edges(data=True):
        d['weight'] = 1
        
    
    #save the graph objects to a list (only suitable if small networks are processed)
    #this is the main objects used for the examples below, which contains all networks
    networks_graphs.append(G)
    labels.append(name)
   

    

    print("loaded", name)

In [ ]:
fh

the graph objects are converted into a list of lists format, which is used by most of the distance functions

In [ ]:
networks = ga.get_node_similarity.preprocess_graph(networks_graphs, attribute="weight")

the networks are now in this format

in each sublists item 0 & 1 are node ids and the edge weight is at position 2

In [ ]:
networks[0]

OPTIONAL: Convert the node names to ints

make sure you save the mapping file to retrieve your original node ids

In [ ]:
network_lists, mapping = ga.get_node_similarity.preprocess_node_list(networks)

In [ ]:
import pickle

In [ ]:
with open(location + "node_id_mapping.pckl", "wb") as f:
    pickle.dump(mapping, f, protocol=4)

# Node Similarity

compute node properties such as degree centrality, betweenness centrality and closeness centrality and shared nodes between networks (when you have networks with different nodes) 

sorted_nodes contains the node ids sorted after the selected properties as well as the mean and median ranking

centrality_values contains the selected properties for each node

shared_nodes contains for each nodes in how many networks it occures (this may be useful for networks that do not have the same nodes)

binary contains the shared nodes in a binary representation

you can select which format is the most suitable for you analysis

make sure to remove asynchrone option when running in jupyter notebooks

this is a wrapper function - if you are only interested in one output you can call the single function directly, for this please refer to the documentation

In [ ]:
sorted_nodes, shared_nodes, binary, centrality_values = ga.get_node_similarity.sort_list_and_get_shared(network_lists, mapping, networks_graphs, labels, degree_centrality=True, closeness_centrality=True, betweenness=True, degree=False, in_async=False)

TODO maybe use kendall instead of wrapper since others are not used/ applicable in this case??

based on this data now multiple distances between the networks can be calculated

this is a wrapper function calling different distance measures, if you are interested in only one distance you can use the single function directly. For this refer to the documentation

this wrapper returns distance/similarity matrices of jaccard similarity, jaccard distance, percentage of shared nodes, kendall rank correlation of degree centrality, closeness centrality and betweenness centrality and mean / median ranking as well as the corresponding p-values for the top and bottom kendall_x nodes (or as here all nodes), hamming distance and SMC similarity

if not all values are needed to be calculated you can use the individual functions called in the wrapper

In [ ]:
j, jd, percentage, kendall_dc_top, b_dc_top, kendall_cc_top, b_cc_top, kendall_betweenness_top, b_b_top, kendall_avg_top, b_avg_top, hamming, kendall_dc_bottom , b_dc_bottom , kendall_cc_bottom , b_cc_bottom , kendall_betweenness_bottom , b_b_bottom , kendall_avg_bottom , b_avg_bottom , smc, kendall_med_top, b_med_top, kendall_med_bottom, b_med_bottom = ga.get_node_similarity.estimate_similarities_nodes(network_lists, sorted_nodes, binary,  kendall_x=len(mapping), is_file=False, in_async=False)

This distances can be merged into a single distance matrix or used individually

Since all networks have the same nodes, we will only use the average rank correlation matrix but transform it into a distance - the correlation value c is transformed to a distance with (1-c)/2 

In [ ]:
import numpy as np

In [ ]:
median_dist = kendall_med_top.copy()

for index, x in np.ndenumerate(median_dist):
    d = (1-x)/2
    
    median_dist[index[0]][index[1]] = d
    
    if index[0] == index[1]:
        median_dist[index[0]][index[1]] = 0

In [ ]:
median_dist_nodes = median_dist.copy()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))  

sns.heatmap(median_dist, annot=False, ax=ax)

# Edge Similarity

we will use the same preprocessed networks as already generated

since the here used networks are unweighted we can assigne each edge its edge betweenness value as weight and compare the networks based on this value for shared edges

In [ ]:
print("sort edges after edge betweenness")
bet = []
graphs_with_betweenness = []
for net in networks_graphs:
    edges_betweenness = nx.edge_betweenness_centrality(net)
    bet.append(edges_betweenness)
    #write as new attribute to graph
    temp = nx.set_edge_attributes(net, edges_betweenness, "betweenness")
    

we need to convert the networks again into the list of lists format, since this time the betweeness values will be used and assign each edge an id

In [ ]:
networks = ga.get_edge_similarity.preprocess_graph(networks_graphs, attribute="betweenness")

print("map edges to id")

network_lists, mapping = ga.get_edge_similarity.preprocess_edge_list(networks)

with open(location + "edge_id_mapping.pckl", "wb") as f:
    pickle.dump(mapping, f, protocol=4)

sort edges after betweenness value & estimate for each edge in which network it appears

In [ ]:
sorted_networks, shared_edges, binary = ga.get_edge_similarity.sort_list_and_get_shared(networks, mapping, network_lists, labels, in_async=False)

compute distances/ similarities between the networks based on their similarity in edges

the wrapper function returns jaccard similarity, jaccard distance, kendall rank coefficient for kendall_x top and bottom edges (here ranked after betweenness), hamming distance and SMC similarity

if only specific distances are needed the individual functions can be called, please refer to the documentation for this

In [ ]:
j, jd, percentage, kendall_top,b_top, kendall_bottom, b_bottom, hamming, smc = ga.get_edge_similarity.estimate_similarities_edges(network_lists, sorted_networks, binary,  kendall_x=100, is_file=False, in_async=False)

convert the similairty measures into distances where applicable

since the networks have different edges, we will use all metrics to create a combined distance matrix (if your network has partially different nodes, you can use these commands to merge & convert them as well)

In [ ]:
smc_dist = smc.copy()

for index, x in np.ndenumerate(smc_dist):
    d = 1-x
    
    smc_dist[index[0]][index[1]] = d
    
    if index[0] == index[1]:
        smc_dist[index[0]][index[1]] = 0

In [ ]:
p_dist = percentage.copy()

for index, x in np.ndenumerate(p_dist):
    d = 1-x
    
    p_dist[index[0]][index[1]] = d
    
    if index[0] == index[1]:
        p_dist[index[0]][index[1]] = 0

In [ ]:
k_dist = kendall_top.copy()

for index, x in np.ndenumerate(k_dist):
    d = (1-x)/2
    
    k_dist[index[0]][index[1]] = d
    
    if index[0] == index[1]:
        k_dist[index[0]][index[1]] = 0

In [ ]:
import statistics

OPTIONAL merge all metrics into a median distance matrix

In [ ]:
median_dist = ga.clustering.create_median_distance_matrix([k_dist, jd, hamming, p_dist, smc_dist], set_diagonal = True)

In [ ]:
median_dist_edges = median_dist.copy()

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))  

sns.heatmap(median_dist, annot=False, ax=ax)

# Structural Similarity

here we compute networ distances based on the graph structure alone

this is a wrapper function that computes a vector based on a few structural parameters implemented

you can create your own vector by creating your own wrapper function or call the corresponding functions


the vector will contain data about the number of nodes & edges, network density, amount of missing edges, cycles, shortest path distributions, clustering coefficient, degree centrality/ closeness centrality and betweenness centrality distribution


some paramters can be expensive on large networks

as input a list of NetworkX graph objects has to be provided

In [ ]:
vectors = ga.get_network_structural_vector.estimate_vector(networks_graphs, edge_attribute="weight", is_file=False)

based on these vectors a distance matrix between the networks can be estimated

there is a wrapper function which estimates the euclidean, canberra, correlation, cosine and jaccard distance based on the vectors (here distance metrics that are not in [0,1] are normalized

In [ ]:
euclidean, canberra, correlation, cosine, jaccard = ga.get_network_structural_vector.matrix_from_vector(vectors, normalize=True)

OPTIONAL: merge the distances into a single distance matrix

In [ ]:
median_dist = ga.clustering.create_median_distance_matrix([jaccard, euclidean, canberra, cosine], set_diagonal = True)

In [ ]:
median_dist_structural = median_dist.copy()

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))  

sns.heatmap(median_dist, annot=False, ax=ax, xticklabels=labels, yticklabels=labels)

# Random Walks

This method aims at characterizing the structural/ connectivity similarities around a specific node in different networks.
Are the same nodes connected?

Perform random walks based on differen starting nodes. Later walks between networks for the same starting node will be compared.

The here used networks have all the same nodes, so nodes can simply be set to the node object of one of the graphs.

E.g. like this:
nodes = networks_graphs[0].nodes()

If the networks have different nodes then you can select which nodes should be compared. E.g. the union of nodes or only their intersection? Or to reduce computational power you can investigate only a subset of pre-selected nodes of interest.

The example below uses the union.

In [ ]:
nodes = []
for net in networks_graphs:
    for node in net.nodes():
        if node not in nodes:
            nodes.append(node)

For each node in nodes 3 times node degree random walks of length 10 are performed. Edges are selected probabilistc based on their edge weight, which is assumed to be a similarity. If no edge weights are existing then all are considered equal. Since the networks used here have all the same edge weight, probabilistic is set to False.

In [ ]:
performed_walks = ga.get_walk_distances.helper_walks(networks_graphs, nodes, labels, steps=10, number_of_walks=3, degree=True, probabilistic=False, weight ="weight")

Now we are estimating for each starting node how often surrounding nodes/ edges have been visit w.r.t. all the visited nodes/ edges. Depending on your network sizes and selected nodes this can be quite memory intensive.

In [ ]:
node_counts, edge_counts, nodes_frc, edges_frc = ga.get_walk_distances.helper_get_counts(labels, networks_graphs, performed_walks)

Now we want to estimate network similarities based on the visited nodes. For each network pair, kendall rank correlation is calculated (of the top 50 nodes)  for the same starting node. The mean correlation value of all same node pairs for a network pair is estimated and returned.

In [ ]:
results_edges, results_nodes, results_edges_p, results_nodes_p = ga.get_walk_distances.helper_walk_sim(networks_graphs, performed_walks, nodes, labels, top=50, undirected=False, return_all = False, nodes_ranked=nodes_frc, edges_ranked=edges_frc)

OPTIONAL: convert the correlation into a distance

In [ ]:
cor_nodes = results_nodes.copy()

for index, x in np.ndenumerate(cor_nodes):
    d = (1-x)/2
    
    cor_nodes[index[0]][index[1]] = d
    
    if index[0] == index[1]:
        cor_nodes[index[0]][index[1]] = 0

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))  

sns.heatmap(cor_nodes, annot=False, ax=ax)

In [ ]:
cor_edges = results_edges.copy()

for index, x in np.ndenumerate(cor_edges):
    d = (1-x)/2
    
    cor_edges[index[0]][index[1]] = d
    
    if index[0] == index[1]:
        cor_edges[index[0]][index[1]] = 0

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))  

sns.heatmap(cor_edges, annot=False, ax=ax)

In [ ]:
median_dist = ga.clustering.create_median_distance_matrix([cor_nodes, cor_edges], set_diagonal = True)

In [ ]:
median_dist_walks = median_dist.copy()

# Clustering

on each of the three median distance matrices, clustering algorithms are run (here we use three algorithms, but this can be adjusted to any algorithms that you prefer for your type of analysis)

based on the individual clusterings a consensus clustering is created

each clustering algorithm is tuned based on a individually modifiable multiobjective function

In [ ]:
distances = [median_dist_nodes, median_dist_edges, median_dist_structural, median_dist_walks]
distances_name = ["nodes", "edges", "structural", "walks"]

In [ ]:
clusterings = {}

for n in distances_name:
    clusterings[n] = []

Hierarchical clustering is run and best k value is estimated on a multiobjective function, focusing on maximizing distance between clusters, minimizing distance within a cluster as well as having an even cluster size distribution
For the best k value the algorithm is run 10 times (One of the 3 algorithms has some randomness. In order to not bias towards one algorithm all are run the same amount of time)

In [ ]:
t = []
for i in range(2,len(labels)):
    t.append(i)
    
hierarchical = {}
for d in range(len(distances)):
    dist = distances[d]
    n = distances_name[d]
    maxs = 10000

    for i, k in enumerate(t):
    #for i, k in enumerate([2, 3]):




        cl_labels = ga.clustering.hierarchical_clustering(dist, n_clusters=k, linkage="complete")


        #print("obj 1")
        avg_score = ga.clustering.multiobjective(dist, cl_labels, min_number_clusters=None, max_number_clusters=None, min_cluster_size = None, max_cluster_size=None, local =True, bet=False, e=None, s=None, cluster_size_distribution = True)



        if avg_score < maxs:
            maxs = avg_score
            mk = k





    hierarchical[n] = k
    print(maxs, mk)
    
    print("creating clusterings for", n, "with k ", mk)
    
    for xx in range(10):
    
    
        cl_labels = ga.clustering.hierarchical_clustering(dist, n_clusters=mk, linkage="complete")
        clusterings.setdefault(n, []).append(cl_labels)



Affinity propagation has no parameters to be set so does not need to be tuned

it is also run 10x (or 10x the same clustering is appended) so that for the later consensus for each clustering algorithm the same number of clusterings are provided

In [ ]:
for d in range(len(distances)):
    dist = distances[d]
    n = distances_name[d]
    
    for xx in range(10):
        cl_labels = ga.clustering.affinityPropagation_clustering(dist)
        clusterings.setdefault(n, []).append(cl_labels)

K Mediods is tuned on the same multiobjective function and for the best k the algorithm is run 3 times

In [ ]:
t = []
for i in range(2,len(labels)):
    t.append(i)
    
kmed = {}
for d in range(len(distances)):
    dist = distances[d]
    n = distances_name[d]
    maxs = 10000

    for i, k in enumerate(t):
    #for i, k in enumerate([2, 3]):




        cl_labels, mediods = ga.clustering.kmedoids_clustering(dist, n_clusters=k)


        #print("obj 1")
        avg_score = ga.clustering.multiobjective(dist, cl_labels, min_number_clusters=None, max_number_clusters=None, min_cluster_size = None, max_cluster_size=None, local =True, bet=False, e=None, s=None, cluster_size_distribution = True)



        if avg_score < maxs:
            maxs = avg_score
            mk = k





    kmed[n] = k
    print(maxs, mk)
    
    print("creating clusterings for", n, "with k ", mk)
    
    for xx in range(10):
    
    
        cl_labels, mediods = ga.clustering.kmedoids_clustering(dist, n_clusters=mk)
        clusterings.setdefault(n, []).append(np.array(cl_labels))



Consensus Clustering is created from the individually performed clusterings

In [ ]:
merged_clusterings = []

for key in clusterings.keys():
    for c in clusterings[key]:
        
        merged_clusterings.append(c.tolist())

Out of all clusterings an agreement graph is created.

Refer to the documentation for other parameter selections

In [ ]:
consensus = ga.clustering.consensus_clustering(merged_clusterings, seed=1234, threshold="matrix", per_node=False, rep = 10)

In [ ]:
consensus

The consensus yields x clusters
By tweaking the treshold or using another method it is possible to tune the consensus based on your needs (you can also use the multiobjective function again to evaluate the consensus numerically)

In [ ]:
max(consensus)+1

In [ ]:
df = pd.DataFrame(list(zip(labels, consensus)), 
               columns =['CHEMICAL', 'CLUSTER'])

In [ ]:
df

save for later examples

TODO upload to git + all other intermediate results

In [ ]:
df.to_csv(location+"clustering_networks.csv", index=None)